# Tutorial

This notebook serves as a tutorial for how to use the Single Particle Model Code. First, begin by running the next cell in order to import all necessary packages for executing the code within this jupyter notebook.

In [1]:
# Import necessary packages
%matplotlib tk
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label
from netCDF4 import Dataset
import netCDF4 as NC
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib import animation
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import (StrMethodFormatter, AutoMinorLocator)
from matplotlib.animation import FuncAnimation

$\bf{\mbox{Generating the input file}}$

The following cells set the input parameters required for the half-SPM simulation.
A $\bf{\mbox{new simulation}}$ can be started by setting 'Starting from checkpoint file' to 'No' and setting the input parameters in the following cells. A NetCDF file called 'SPM_input.nc' will automatically be created which is then read by the programme.

The simulation can also be $\bf{\mbox{continued from a checkpoint}}$ file that is created during the first run by setting 'Starting from checkpoint file' to 'yes'. In that case, $\bf{\mbox{only the next two cells have to be run}}$ (provided the original 'input.nc' file still exists) and no further parameters have to be set.

In [3]:
Checkpoint_ =  HBox([Label('Starting from checkpoint file'), widgets.Select(options=['Yes', 'No'], value='No', disabled=False)])
Sim_steps_c = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Out_steps_c = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
display(Checkpoint_, Sim_steps_c, Out_steps_c)

In [4]:
# Checkpoint check
if Checkpoint_.children[1].value == 'Yes':
        Checkpoint = 1
        rootgrp = NC('SPM_input.nc', 'r+', format='NETCDF4')
        rootgrp['checkpoint'][:] = Checkpoint
        rootgrp['sim_steps'][:] = Sim_steps_c.children[1].value 
        rootgrp['out_steps'][:] = Out_steps_c.children[1].value 
        rootgrp.close()
else:
        Checkpoint = 0

The following cell allow to set the input parameters within a suggested range if a new simulation run is set up. The current default values are taken from Chen2020 and for a NCM positive electrode.

There is the option to set the input parameters unreseticted in the following cell. This is only recommended for experienced users and only physical valid values should be set.

The remaining cells extract the set input parameters, convert them to SI units and creates a NetCDF input file that can be read by the programme.

In [2]:
# set input parameter within given ranges
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatSlider(min=-20.0, max=50.0, value=21.0, step=1)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatSlider(min=3.22, max=7.22, value=5.22, step=0.01)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatSlider(min=50.0, max=100.0, value=75.6, step=0.1)])
Rr_coef_ = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatSlider(min=1.5, max=6.5, value=3.42, step=0.01)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatSlider(min=0, max=100, value=1.48, step=0.01)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatSlider(min=0.0, max=100000.0, value=51765.0, step=1)])
SOC_ = HBox([Label('State of charge [%]'), widgets.FloatSlider(min=0, max=100, value=0, step=1)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatSlider(min=-20.0, max=20.0, value=5.0, step=0.1)])
Vol_per_ = HBox([Label('Active material volume fraction [%]'), widgets.FloatSlider(min = 0.0, max=100.0, value=66.5, step=0.1)])
Area_ = HBox([Label('Electrode plate area [$m^2$]'), widgets.FloatSlider(min=0.001, max=1.0, value=0.1027, step=0.001)])
Sim_steps_ = HBox([Label('No. of simulation steps'), widgets.BoundedIntText(value=1000, min=1, max=10**9, disabled=False)])
Dt_ = HBox([Label('Time step (s)'), widgets.FloatSlider(min=0.01, max=100.0, value=2.0, step=0.01)]) 
Out_steps_ = HBox([Label('Output written every [n] steps'), widgets.BoundedIntText(value=5, min=1, max=10**9, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatSlider(min=2, max=1000, value=20, step=1)])
Volt_do_ =  HBox([Label('Output voltage data'), widgets.Select(options=['Yes', 'No'], value='Yes', disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, SOC_, Current_, Vol_per_, Area_, Sim_steps_, Dt_, Out_steps_, Space_steps_, Volt_do_)

In [ ]:
# unbounded boxes to set paramters outside of suggested ranges if wanted
Temp_ = HBox([Label('Temperature [°C]'), widgets.FloatText(value=Temp_.children[1].value, disabled=False)])
Rad_ = HBox([Label('Mean particle radius [$\mu m$]'), widgets.FloatText(value=Rad_.children[1].value, disabled=False)])
Thick_ = HBox([Label('Electrode thickness [$\mu m$]'), widgets.FloatText(value=Thick_.children[1].value, disabled=False)])
Rr_coef_ = HBox([Label('Reaction rate coeffic[ent($Am^{-2}(m^3mol^{-1})^{1.5}$]'), widgets.FloatText(value=Rr_coef_.children[1].value, disabled=False)])
Dif_coef_ = HBox([Label('Diffusion coefficient [$10^{-15} m^2 s^{-1}$]'), widgets.FloatText(value=Dif_coef_.children[1].value, disabled=False)])
Max_c_ = HBox([Label('Maximum lithium concentration [$mol m^{-3}$]'), widgets.FloatText(value=Max_c_.children[1].value, disabled=False)])
Current_ = HBox([Label('Applied current [A]'), widgets.FloatText(value=Current_.children[1].value, disabled=False)])
Area_ = HBox([Label('Electrode area [$m^2$]'), widgets.FloatText(value=Area_.children[1].value, disabled=False)])
Dt_ = HBox([Label('Time step (s)'), widgets.FloatText(value=Dt_.children[1].value, disabled=False)])
Space_steps_ = HBox([Label('No. of space steps'), widgets.FloatText(value=Space_steps_.children[1].value, disabled=False)])

display(Temp_, Rad_, Thick_, Rr_coef_, Dif_coef_, Max_c_, Current_, Area_, Dt_, Space_steps_)

In [5]:
# saving the values set y the user and adjusting relevant parameters to be in SI units
Temp = Temp_.children[1].value + 273.15   #K
Rad = Rad_.children[1].value * 10**(-6)   #m
Thick = Thick_.children[1].value * 10**(-6)   #m
Rr_coef =  Rr_coef_.children[1].value
Dif_coef = Dif_coef_.children[1].value * 10**(-15)
Max_c = Max_c_.children[1].value
# Init_c derived from values in Chen2020:
# stoichiometry at 0% SOC = 0.2661, and at 100% SOC = 0.9084
# assuming linear behaviour we get x = mcx +c with c = 0.9084
# and m = 0.2661-0.9084 = -0.6423
Init_c = Max_c * ((SOC_.children[1].value/100)*(-0.6423) + 0.9084)
Iapp = Current_.children[1].value / Area_.children[1].value  # A/m^2
Vol_per = Vol_per_.children[1].value
Sim_steps = Sim_steps_.children[1].value
Dt = Dt_.children[1].value
Out_steps = Out_steps_.children[1].value
Space_steps = Space_steps_.children[1].value
if Volt_do_.children[1].value == 'Yes':
    Volt_do = 1
else:
    Volt_do = 0

In [19]:
#create new NetCDF file in 'writing mode' and 'NETCDF4 format'
rootgrp = Dataset('SPM_input.nc', 'w', format='NETCDF4')
#creating dimensions for vector holding all the input variables
temp_dim = rootgrp.createDimension('temp_dim', 1)
rad_dim = rootgrp.createDimension('rad_dim', 1)
thick_dim = rootgrp.createDimension('thick_dim', 1)
rr_coef_dim = rootgrp.createDimension('rr_coef_dim', 1)
dif_coef_dim = rootgrp.createDimension('dif_coef_dim', 1)
max_c_dim = rootgrp.createDimension('max_c_dim', 1)
init_c_dim = rootgrp.createDimension('init_c_dim', 1)
iapp_dim = rootgrp.createDimension('iapp_dim', 1)
vol_per_dim = rootgrp.createDimension('vol_per_dim', 1)
sim_steps_dim = rootgrp.createDimension('sim_steps_dim', 1)
dt_dim = rootgrp.createDimension('dt_dim', 1)
out_steps_dim = rootgrp.createDimension('out_steps_dim', 1)
space_steps_dim = rootgrp.createDimension('space_steps_dim', 1)
volt_do_dim = rootgrp.createDimension('volt_do_dim', 1)
checkpoint_dim = rootgrp.createDimension('checkpoint_dim', 1)
#createing variable 
temp = rootgrp.createVariable('temp', 'f8', ('temp_dim',))
rad = rootgrp.createVariable('rad', 'f8', ('rad_dim',))
thick = rootgrp.createVariable('thick', 'f8', ('thick_dim',))
rr_coef = rootgrp.createVariable('rr_coef', 'f8', ('rr_coef_dim',))
dif_coef = rootgrp.createVariable('dif_coef', 'f8', ('dif_coef_dim',))
max_c = rootgrp.createVariable('max_c', 'f8', ('max_c_dim',))
init_c = rootgrp.createVariable('init_c', 'f8', ('init_c_dim',))
iapp = rootgrp.createVariable('iapp', 'f8', ('iapp_dim',))
vol_per = rootgrp.createVariable('vol_per', 'f8', ('vol_per_dim',))
sim_steps = rootgrp.createVariable('sim_steps', 'i4', ('sim_steps_dim',))
dt = rootgrp.createVariable('dt', 'f8', ('dt_dim',))
out_steps = rootgrp.createVariable('out_steps', 'i4', ('out_steps_dim',))
space_steps = rootgrp.createVariable('space_steps', 'i4', ('space_steps_dim',))
volt_do = rootgrp.createVariable('volt_do', 'i4', ('volt_do_dim',))
checkpoint = rootgrp.createVariable('checkpoint', 'i4', ('checkpoint_dim',))
# attributes
rootgrp.description = 'Input parameters for SMP model'
temp.description = 'Temperature'
temp.units = 'K'
rad.description = 'Mean particle radius'
rad.units = 'm'
thick.description = 'Electrode thickness'
thick.units = 'm'
rr_coef.description = 'Reaction rate coefficient'
rr_coef.units = '$Am^{-2}(m^3mol^{-1})^{1.5}$'
dif_coef.description ='Diffusion coefficient'
dif_coef.units = '$m^2 s^{-1}$'
max_c.description = 'Maximum lithium concentration'
max_c.units = '$mol m^{-3}$'
init_c.description = 'Initial lithium concentration'
init_c.units = '$mol m^{-3}$'
iapp.description = 'Applied current density'
iapp.units = '$A/m^2$'
vol_per.description = 'Active material volume fraction'
vol_per.units = '%'
sim_steps.description = 'Total number of simultion steps'
sim_steps.units = 'unitless'
dt.description = 'Time step'
dt.units = 's'
out_steps.description = 'Output written every [n] number of steps'
out_steps.units = 'untiless'
space_steps.description = 'No. of space steps'
space_steps.units = 'unitless'
volt_do.description = 'Write voltage data'
volt_do.units = 'unitless'
checkpoint.description = 'Starting from checkpont file'
checkpoint.units = 'unitless'
# writing data to input_parameters variable
temp[0] = Temp
rad[0] = Rad
thick[0] = Thick
rr_coef[0] = Rr_coef
dif_coef[0] = Dif_coef
max_c[0] = Max_c
init_c[0] = Init_c
iapp[0] = Iapp
vol_per[0] = Vol_per
sim_steps[0] = Sim_steps
dt[0] = Dt
out_steps[0] = Out_steps
space_steps[0] = Space_steps
volt_do[0] = Volt_do
checkpoint[0] = Checkpoint
#closing the NetCDF file
rootgrp.close()

$\bf{\mbox{Compiling and running the main program}}$

The following cells are used to compile all relevant files and the run the main program. The first cell executes a terminal command within the directory this file is located. In this case, the cell runs the "make" command which compiles all relevent files. The following cell executes "make exe" which runs the executable created by the first cell. The laste cell of this section executes "make clean" which deletes previously generated files and allows for the first cell to be run again. This cell should only be run if there are any errors with the previous cells.

In [20]:
# Run this cell to compile the code and create the executables
!make

gfortran  `nf-config --fflags`  -c input_output_netcdf.f90 `nf-config --flibs` -llapack  -o input_output_netcdf.o
gfortran  `nf-config --fflags`  -c pde.f90 `nf-config --flibs` -llapack  -o pde.o
gfortran  `nf-config --fflags`  input_output_netcdf.o pde.o main.f90 `nf-config --flibs` -llapack  -o test.out
rm -f *.o *.mod
Crank-Nicolson solver ready
Input successful
Compilation sucessful


In [22]:
# Run this cell to execute the code and generate data
!make exe

Running Serial code


In [12]:
# If errors are being displayed, run this cell to delete all files created by make. 
# Next, run the first and second cells of this section again
!make clean

rm -f *.o *.mod *.nc *chp test.out
rm -f uq_code/*.nc uq_code/*.csv
rm -r uq_code/data_store_sens
rm: cannot remove 'uq_code/data_store_sens': No such file or directory
make: [Makefile:83: clean] Error 1 (ignored)
Files removed


$\bf{\mbox{Plotting}}$

Run the cell below to display the animation of the data that was generated. The cell displays plots of Lithium concentration across the particle radius as a function of time. It also displays the concentration across the particle as a coloured contour plot as well as plot of the ouput voltage against time step.

In [31]:
# Creates a pop up window with the plotted data
plt.close()
# reading in the NetCDF data from SP_output.nc
dat = NC.Dataset("SP_output.nc", "r", format="NETCDF4")
do_volt = dat['volt_do'][:][0]

#creating figure and axes dependent on whether voltage data is to be written or not
if do_volt == 1:
	fig = plt.figure(figsize=(10,8))
	ax1 = plt.subplot2grid((2,2),(0,0))
	ax2 = plt.subplot2grid((2,2),(0,1))
	ax3 = plt.subplot2grid((2,2),(1,0), colspan=2)
else:
	fig = plt.figure(figsize=(10,4))
	ax1 = plt.subplot2grid((1,2),(0,0))
	ax2 = plt.subplot2grid((1,2),(0,1))


# Subplot 1 - Animtion of lithium concentration 
# accessing the 2d concentration data and creating a time and space axis
c = dat['conc'][:]

t_steps = np.shape(c)[0]
r_steps = dat['space_steps'][:][0]
rad = dat['rad'][:][0]*10**(6)
time_axis = np.linspace(0, t_steps-1, t_steps-1).astype(int)
x_axis = np.linspace(0, rad, r_steps, endpoint=True)

# animation of the concentration data 
# the main parts of the  animation code were taken from:
# https://brushingupscience.com/2016/06/21/matplotlib-animations-the-easy-way/

intervaltime = 0.5   #time between each animations step in ms

# plotting the first graph which is a 2D line of lithium ion concentration
# with respect to the particle radius
line, = ax1.plot(x_axis, c[0])

# function evolving the animation, which is called each frame 
# the x-axis is the particle radius, so only the x-axis data
# i.e. the lithium concantration changes with simulation time
def animate(t):
    line.set_ydata(c[t,:])
    return line,

# creating the animation using matplotlib's FunAnimation
# the arguments to be passed:
# figure of the graph  - fig from fix,ax = plt.subplots())
# function evolving the animation - animate(t)
# interval time between each timesetep - intervaltime
# frames, the array to iterate over - time_axis
# blit - blitting set to true
animation = FuncAnimation(fig, animate, interval=intervaltime, frames=time_axis, blit=True)

# customise the graph with axes labels, major and minor ticks, labels etc.
ax1.set_xlabel('Distance from particle centre [$\mu m$]', size=8)
ax1.xaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(axis='x', labelsize=7)
ax1.set_ylabel('Lithium concentration $mol*m^{-3}$', size=8)
ax1.set_ylim(np.min(c)-1*10**(-9), np.max(c)+1*10**(-9))
ax1.yaxis.set_minor_locator(AutoMinorLocator())
ax1.tick_params(axis='y', labelsize=7)
ax1.ticklabel_format(axis='both', style="sci", useMathText=True)
ax1.xaxis.offsetText.set_fontsize(7)
ax1.yaxis.offsetText.set_fontsize(7)
ax1.set_title('Lithium Concentration Across Particle Radius', size=10, pad=15.0)

# Subplot 2 - pcolorplot of concentration data

# most of the required data is the same as in subplot 1
# get the discretisation steps of the radius
dr = rad/(r_steps-1)
 
# reversed virdis colormap, values over the range are black, under goes white
colour = cm.get_cmap('viridis_r').copy()
colour.set_under(color='w')
colour.set_over(color='k')
max_colour = np.max(c)
min_colour = np.min(c)
ticklist = np.linspace(c.min(), c.max(), 6)

#create grid for plotting
x = np.linspace(-rad,rad,r_steps,endpoint=True)
y = np.linspace(-rad,rad,r_steps,endpoint=True)

# form 2D array of radii
z = np.array([np.sqrt(i*i+j*j) for j in y for i in x])

# form concentration values; give negative concentration for colourmap if out of range - goes white
for idx, r in enumerate(z):
	if (r <= rad):
		ri = np.rint(r/dr).astype(np.int32)
		z[idx] = c[0,:][ri]
	else:
		z[idx] = -1

# reshape to form 2D array for pcolor
Z = z.reshape(r_steps,r_steps)

# form meshgrid and plot using pcolor
X, Y = np.meshgrid(x,y)
cplot = ax2.pcolor(X,Y,Z, cmap=colour, vmin=min_colour, vmax=max_colour, edgecolors='face')
ax2.set_xlabel('Distance from particle centre [$\mu m$]', size=8)
ax2.xaxis.set_minor_locator(AutoMinorLocator())
ax2.tick_params(axis='x', labelsize=7)
ax2.set_ylabel('Distance from particle centre [$\mu m$]', size=8)
ax2.yaxis.set_minor_locator(AutoMinorLocator())
ax2.tick_params(axis='y', labelsize=7)
ax2.ticklabel_format(axis='both', style="sci", useMathText=True)
ax2.xaxis.offsetText.set_fontsize(7)
ax2.yaxis.offsetText.set_fontsize(7)
ax2.set_title('Contour Plot of Concentration Inside Particle', size=10, pad=15.0)
cbar = ax2.figure.colorbar(cplot, ax=ax2, cmap=colour, ticks=ticklist)
cbar.ax.tick_params(labelsize=7)
cbar.ax.set_ylabel('Lithium Concentration', size=8)
cbar.ax.yaxis.set_major_formatter(StrMethodFormatter("{x:.10f}"))

# animation function for the pcolor plot animation
# works generally the same as for subplot 1
# the concentration data for each timestep is created as in the pcolor plot above
# while iterating through the time dimension of the concentratoin data
# the resulting 2D array is flattened in the end to an 1D array that can be used by FuncAnimation
# the inteval and frames are the same as in subplot 1 and blitting is set to true
def animate_pcol(t):
    z =  np.array([np.sqrt(i*i+j*j) for j in y for i in x])
    for idx, r in enumerate(z):
	    if (r <= rad):
		    ri = np.rint(r/dr).astype(np.int32)
		    z[idx] = c[t,:][ri]
	    else:
		    z[idx] = -1
    Z = z.reshape(r_steps,r_steps)
    cplot.set_array(Z.flatten())
    return cplot,

animation_pcol = FuncAnimation(fig, animate_pcol, interval=intervaltime, frames=time_axis, blit=True)

# Subplot 3 - plot of voltage output if do_volt set to true
if do_volt == 1:

	# accessing the voltage data and setting the number of time steps as x-axis
	volt = dat['volt'][:][:,0]
	time = np.linspace(0, t_steps, t_steps)

	# plotting the 2D graph and customising it
	ax3.plot(time*Dt, volt)
	ax3.set_xlabel('Time [s]', size=8)
	ax3.xaxis.set_minor_locator(AutoMinorLocator())
	ax3.tick_params(axis='x', labelsize=7)
	ax3.set_ylabel('Voltage [V]', size=8)
	ax3.yaxis.set_minor_locator(AutoMinorLocator())
	ax3.tick_params(axis='y', labelsize=7)
	ax3.ticklabel_format(axis='both', style="sci", useMathText=True)
	ax3.xaxis.offsetText.set_fontsize(7)
	ax3.yaxis.offsetText.set_fontsize(7)
	ax3.set_title('Voltage Output', size=10, pad=15.0)

	# animation of an curser that moves with the time axis with the same interval and frames
	# as the other two animations
	vl = ax3.axvline(time[0], color='black', linestyle=':')
	ax3.set_xlim()

	def animate_t_bar(t):
		vl.set_xdata(time[t])
		return vl,

	animation_t_bar = FuncAnimation(fig, animate_t_bar, interval=intervaltime, frames=time_axis, blit=True)

plt.draw()
plt.tight_layout()
plt.show()

dat.close()